# Stochastic Gradient Descent (SGD)

Gradient descent computes gradients for each instance in the training set.
This can be expensive for large datasets.
Note that can take a random subset 
$\mathcal{B} \subset \mathcal{D}$ such that $B = |\mathcal{B}| \ll |\mathcal{D}|$
and still get an unbiased estimate $\mathcal{L}_{\mathcal{B}} \approx \mathcal{L}_\mathcal{D}$
of the empirical loss surface. 
This method is called **Stochastic Gradient Descent** (SGD). This makes sense since $\mathcal{L}_\mathcal{D}$ is also an estimate of the true loss surface that is fixed at each training step.
SGD a lot cheaper to compute compared to batch GD allowing training to progress faster with more updates.
Moreover, SGD has been shown to escape saddle points with some theoretical guarantees {cite}`dauphin2014`.
The update rule for SGD is given by:

$$
\boldsymbol{\Theta}^{t+1} = \boldsymbol{\Theta}^t - \eta\; \nabla_{\boldsymbol{\Theta}}\, \mathcal L_{\mathcal{B}}(\boldsymbol{\Theta}^t)
$$

Typically, we take $B = 8, 32, 64, 128, 256.$ Note that SGD is essentially GD above it just replaces the function $f$ at each step with $\mathcal{L}_{\mathcal{B}}.$ Hence, all modifications of GD discussed have the same update rule for SGD. 
The same results and observations also mostly hold. Although, now we have to reason with noisy approximations $f_t \approx f$ at each step unlike before where it is fixed.

In [1]:
from chapter import *

In [2]:
from functools import partial

def loss(w0, w1, X, y):
    return ((X @ np.array([w0, w1]) - y)**2).mean()

def grad(w, X, y, B=None):
    """Gradient step for the MSE loss function"""
    dw = 2*((X @ w - y).reshape(-1, 1) * X).mean(axis=0)
    return dw / np.linalg.norm(dw)

def sgd(w0, X, y, eta=0.1, steps=10, B=32):
    """Return sequence of weights from GD."""
    w = np.zeros([steps, 2])
    w[0, :] = w0
    for j in range(1, steps):
        batch = torch.randint(0, len(X), size=(B,))
        u = w[j-1, :]
        w[j, :] = u - eta * grad(u, X[batch], y[batch])
    return w


# Generate data
B = 4
n = 1000
X = np.zeros((n, 2))
X[:, 1] = np.random.uniform(low=-1, high=1, size=n)
X[:, 0] = 1
w_min = np.array([-1, 3])
y = (X @ w_min) + 0.05 * np.random.randn(n)  # data: y = -1 + 3x + noise

# Gradient descent
w_init = [-4, -4]
w_step_gd  = sgd(w_init, X, y, eta=0.5, steps=30, B=len(X))
w_step_sgd = sgd(w_init, X, y, eta=0.5, steps=30, B=B)

In [3]:
# Create a figure and two subplots
fig = plt.figure(figsize=(12, 11))
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223, projection='3d')
ax4 = fig.add_subplot(224, projection='3d')

# Call the functions with the respective axes
plt.tight_layout()
plot_contourf(ax1, partial(loss, X=X, y=y), w_step_gd, color="red")
plot_contourf(ax2, partial(loss, X=X, y=y), w_step_sgd, color="red")
plot_surface(ax3, partial(loss, X=X, y=y), N=50)
for i in range(5):
    batch = torch.randint(0, len(X), size=(B,))
    plot_surface(ax4, f=partial(loss, X=X[batch], y=y[batch]), N=50)

ax1.set_title("GD")
ax2.set_title(f"SGD (B={B})")
ax3.set_title("\nloss surface")
ax4.set_title(f"\nloss approx. (B={B})")
fig.tight_layout()
plt.show()

<br>

**Remark.** Randomly sampling from $N \gg 1$ points at each step is expensive. Instead, we typically just shuffle the dataset once, then iterate over it with slices of size $B$. This is essentially sampling without replacement which turns out that this is more [data efficient](https://www.d2l.ai/chapter_optimization/sgd.html#stochastic-gradients-and-finite-samples) (i.e. the model gets to see more varied data). One such pass over the dataset is called an **epoch**. This is done for example when using PyTorch DataLoaders:

In [4]:
from torch.utils.data import DataLoader

train_loader = DataLoader(torch.arange(10), batch_size=2, shuffle=True)

print("Epoch 1:")
[print(x) for x in train_loader]
print()
print("Epoch 2:")
[print(x) for x in train_loader];

Epoch 1:
tensor([7, 6])
tensor([5, 4])
tensor([9, 1])
tensor([0, 8])
tensor([2, 3])

Epoch 2:
tensor([3, 0])
tensor([5, 2])
tensor([7, 6])
tensor([4, 8])
tensor([1, 9])
